In [1]:
import xarray as xr
import zarr
from numcodecs import Blosc

In [2]:
xr.__version__

'2024.6.0'

## Open Dataset

In this example, I am only using a single one. But you probably want to open multiple files to consolidate.


```python
ds = xr.open_mfdataset(filename, engine='netcdf4')
```

where filename = 'path_to'

In [18]:
path = ''
filename = path+'MITgcm_rect_nc.nc'
filename

'MITgcm_rect_nc.nc'

In [19]:
ds = xr.open_dataset(filename, engine='netcdf4')
ds

<xarray.Dataset> Size: 6MB
Dimensions:     (time_midp: 16, Z: 14, Zp1: 15, Zu: 14, Zl: 14, X: 11, Y: 13,
                 Xp1: 12, Yp1: 14, time: 17)
Coordinates: (12/18)
  * time_midp   (time_midp) datetime64[ns] 128B 2007-09-01T03:00:00 ... 2007-...
  * Z           (Z) float64 112B -1.0 -3.5 -7.0 -11.5 ... -83.5 -97.0 -111.5
  * Zp1         (Zp1) float64 120B 0.0 -2.0 -5.0 -9.0 ... -90.0 -104.0 -119.0
  * Zu          (Zu) float64 112B -2.0 -5.0 -9.0 -14.0 ... -90.0 -104.0 -119.0
  * Zl          (Zl) float64 112B 0.0 -2.0 -5.0 -9.0 ... -77.0 -90.0 -104.0
  * X           (X) float64 88B -17.98 -17.93 -17.89 ... -17.62 -17.58 -17.54
    ...          ...
  * Yp1         (Yp1) float64 112B 70.26 70.28 70.3 70.31 ... 70.47 70.49 70.51
    XV          (Yp1, X) float64 1kB ...
    YV          (Yp1, X) float64 1kB ...
    XG          (Yp1, Xp1) float64 1kB ...
    YG          (Yp1, Xp1) float64 1kB ...
  * time        (time) datetime64[ns] 136B 2007-09-01 ... 2007-09-05
Data variables: (12/90)
    drC         (Zp1) float64 120B ...
    drF         (Z) float64 112B ...
    dxC         (Y, Xp1) float64 1kB ...
    dyC         (Yp1, X) float64 1kB ...
    dxF         (Y, X) float64 1kB ...
    dyF         (Y, X) float64 1kB ...
    ...          ...
    KPPg_SLT    (time_midp, Zl, Y, X) float64 256kB ...
    KPPg_TH     (time_midp, Zl, Y, X) float64 256kB ...
    SFLUX       (time_midp, Y, X) float64 18kB ...
    TFLUX       (time_midp, Y, X) float64 18kB ...
    oceQsw_AVG  (time_midp, Y, X) float64 18kB ...
    oceSPtnd    (time_midp, Z, Y, X) float64 256kB ...
Attributes:
    OceanSpy_grid_coords:    {'Y': {'Y': None, 'Yp1': 0.5}, 'X': {'X': None, ...
    OceanSpy_name:           EGshelfIIseas2km_ASR
    OceanSpy_description:    High-resolution (~2km) numerical simulation cove...
    OceanSpy_parameters:     {'rSphere': 6371.0, 'eq_state': 'jmd95', 'rho0':...
    OceanSpy_projection:     Mercator(**{'central_longitude': -17.49216449999...
    OceanSpy_grid_periodic:  []

In [5]:
ds['Temp']

<xarray.DataArray 'Temp' (time: 17, Z: 14, Y: 13, X: 11)> Size: 272kB
[34034 values with dtype=float64]
Coordinates:
  * Z        (Z) float64 112B -1.0 -3.5 -7.0 -11.5 ... -71.0 -83.5 -97.0 -111.5
  * X        (X) float64 88B -17.98 -17.93 -17.89 ... -17.62 -17.58 -17.54
  * Y        (Y) float64 104B 70.27 70.29 70.3 70.32 ... 70.44 70.46 70.48 70.5
    XC       (Y, X) float64 1kB ...
    YC       (Y, X) float64 1kB ...
  * time     (time) datetime64[ns] 136B 2007-09-01 ... 2007-09-05
Attributes:
    units:            degC
    long_name:        potential_temperature
    original_output:  snapshot
    _coordinates:     XC YC RC iter

## define chunk parameters

* In this very simple example, I am going to just chunk in time. 

In [6]:
ds['Temp'].chunk()

<xarray.DataArray 'Temp' (time: 17, Z: 14, Y: 13, X: 11)> Size: 272kB
dask.array<xarray-<this-array>, shape=(17, 14, 13, 11), dtype=float64, chunksize=(17, 14, 13, 11), chunktype=numpy.ndarray>
Coordinates:
  * Z        (Z) float64 112B -1.0 -3.5 -7.0 -11.5 ... -71.0 -83.5 -97.0 -111.5
  * X        (X) float64 88B -17.98 -17.93 -17.89 ... -17.62 -17.58 -17.54
  * Y        (Y) float64 104B 70.27 70.29 70.3 70.32 ... 70.44 70.46 70.48 70.5
    XC       (Y, X) float64 1kB dask.array<chunksize=(13, 11), meta=np.ndarray>
    YC       (Y, X) float64 1kB dask.array<chunksize=(13, 11), meta=np.ndarray>
  * time     (time) datetime64[ns] 136B 2007-09-01 ... 2007-09-05
Attributes:
    units:            degC
    long_name:        potential_temperature
    original_output:  snapshot
    _coordinates:     XC YC RC iter

In [7]:
data_vars = [var for var in ds.data_vars if 'time' in ds[var].dims]


# The line below rechunks in time.

In [8]:
for var in data_vars:
    ds[var] = ds[var].chunk({'time':1})

## bad example because this is a very small file
* ideally want chunk size O(100 Mb), otherwise performance can be bad if too small like in this case!


In [9]:
ds['Temp']

<xarray.DataArray 'Temp' (time: 17, Z: 14, Y: 13, X: 11)> Size: 272kB
dask.array<xarray-<this-array>, shape=(17, 14, 13, 11), dtype=float64, chunksize=(1, 14, 13, 11), chunktype=numpy.ndarray>
Coordinates:
  * Z        (Z) float64 112B -1.0 -3.5 -7.0 -11.5 ... -71.0 -83.5 -97.0 -111.5
  * X        (X) float64 88B -17.98 -17.93 -17.89 ... -17.62 -17.58 -17.54
  * Y        (Y) float64 104B 70.27 70.29 70.3 70.32 ... 70.44 70.46 70.48 70.5
    XC       (Y, X) float64 1kB ...
    YC       (Y, X) float64 1kB ...
  * time     (time) datetime64[ns] 136B 2007-09-01 ... 2007-09-05
Attributes:
    units:            degC
    long_name:        potential_temperature
    original_output:  snapshot
    _coordinates:     XC YC RC iter

### Define a global compressor and encoding for each variable
* Many zarr files are compressed via Blosc (`clevel=0` is zero compression, `clevel=9` is maximum compression)
* BITSHUFFLE also helps compress data (better compression ratio with data that does not vary much like salinity when compared to Temperature in Celsius)


# Before doing that, I need to drop this variable:

* 'KPPHBL'

Causing me some trouble...?


In [10]:
ds = ds.reset_coords().drop_vars(['KPPHBL'])

In [11]:
compressor = Blosc(cname='zstd', clevel=3, shuffle=Blosc.BITSHUFFLE)
encoding = {vname: {'compressor': compressor} for vname in ds.data_vars}

In [12]:
encoding

{'drC': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'drF': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'XC': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'YC': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'XU': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'YU': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'XV': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'YV': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'XG': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'YG': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'dxC': {'compressor': Blosc(cname='zstd', clevel=3, shuffle=BITSHUFFLE, blocksize=0)},
 'dyC': {'compressor': Blosc(cname='zstd

In [13]:
new_filename = path+'MITgcm_rect'

In [16]:
ds.to_zarr(new_filename, mode='w', encoding=encoding, consolidated=True)

In [17]:
xr.open_zarr(new_filename)

<xarray.Dataset> Size: 6MB
Dimensions:     (time_midp: 16, Zl: 14, Y: 13, X: 11, Z: 14, Xp1: 12, Yp1: 14,
                 time: 17, Zp1: 15, Zu: 14)
Coordinates: (12/18)
  * X           (X) float64 88B -17.98 -17.93 -17.89 ... -17.62 -17.58 -17.54
    XC          (Y, X) float64 1kB dask.array<chunksize=(13, 11), meta=np.ndarray>
    XG          (Yp1, Xp1) float64 1kB dask.array<chunksize=(14, 12), meta=np.ndarray>
    XU          (Y, Xp1) float64 1kB dask.array<chunksize=(13, 12), meta=np.ndarray>
    XV          (Yp1, X) float64 1kB dask.array<chunksize=(14, 11), meta=np.ndarray>
  * Xp1         (Xp1) float64 96B -18.0 -17.95 -17.91 ... -17.6 -17.56 -17.51
    ...          ...
  * Z           (Z) float64 112B -1.0 -3.5 -7.0 -11.5 ... -83.5 -97.0 -111.5
  * Zl          (Zl) float64 112B 0.0 -2.0 -5.0 -9.0 ... -77.0 -90.0 -104.0
  * Zp1         (Zp1) float64 120B 0.0 -2.0 -5.0 -9.0 ... -90.0 -104.0 -119.0
  * Zu          (Zu) float64 112B -2.0 -5.0 -9.0 -14.0 ... -90.0 -104.0 -119.0
  * time        (time) datetime64[ns] 136B 2007-09-01 ... 2007-09-05
  * time_midp   (time_midp) datetime64[ns] 128B 2007-09-01T03:00:00 ... 2007-...
Data variables: (12/89)
    ADVr_SLT    (time_midp, Zl, Y, X) float64 256kB dask.array<chunksize=(16, 14, 13, 11), meta=np.ndarray>
    ADVr_TH     (time_midp, Zl, Y, X) float64 256kB dask.array<chunksize=(16, 14, 13, 11), meta=np.ndarray>
    ADVx_SLT    (time_midp, Z, Y, Xp1) float64 280kB dask.array<chunksize=(8, 14, 13, 12), meta=np.ndarray>
    ADVx_TH     (time_midp, Z, Y, Xp1) float64 280kB dask.array<chunksize=(8, 14, 13, 12), meta=np.ndarray>
    ADVy_SLT    (time_midp, Z, Yp1, X) float64 276kB dask.array<chunksize=(8, 14, 14, 11), meta=np.ndarray>
    ADVy_TH     (time_midp, Z, Yp1, X) float64 276kB dask.array<chunksize=(8, 14, 14, 11), meta=np.ndarray>
    ...          ...
    rA          (Y, X) float64 1kB dask.array<chunksize=(13, 11), meta=np.ndarray>
    rAs         (Yp1, X) float64 1kB dask.array<chunksize=(14, 11), meta=np.ndarray>
    rAw         (Y, Xp1) float64 1kB dask.array<chunksize=(13, 12), meta=np.ndarray>
    rAz         (Yp1, Xp1) float64 1kB dask.array<chunksize=(14, 12), meta=np.ndarray>
    surForcS    (time, Y, X) float64 19kB dask.array<chunksize=(1, 13, 11), meta=np.ndarray>
    surForcT    (time, Y, X) float64 19kB dask.array<chunksize=(1, 13, 11), meta=np.ndarray>
Attributes:
    OceanSpy_description:    High-resolution (~2km) numerical simulation cove...
    OceanSpy_grid_coords:    {'Y': {'Y': None, 'Yp1': 0.5}, 'X': {'X': None, ...
    OceanSpy_grid_periodic:  []
    OceanSpy_name:           EGshelfIIseas2km_ASR
    OceanSpy_parameters:     {'rSphere': 6371.0, 'eq_state': 'jmd95', 'rho0':...
    OceanSpy_projection:     Mercator(**{'central_longitude': -17.49216449999...